# Relabeling

We want to improve the quality of the labels from AMT. In particular, the FPL.

In [3]:
import pickle

import datetime
import pandas as pd

from data.dao import DataAccess, LabelGetter
from data.turk import TurkResults2Label

In [27]:
X = DataAccess.get_as_dataframe()

In [29]:
L = LabelGetter(X)

In [30]:
X_fpl, y = L.get_first_person_label()

In [32]:
X_fpl.shape

(6357, 11)

Deterministic kind of uniform sample:

In [34]:
%%time
X_fpl['random_number'] = X_fpl.text.apply(hash)

Wall time: 9.69 s


C:\Users\Tom Work\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [8]:
X_fpl.head()

,created_at,labels,predict,text,user,user.created_at,user.favourites_count,user.followers_count,user.friends_count,user.statuses_count,user.verified,random_number
_id,,,,,,,,,,,,
556e12e6d6dfbb462881153e,Tue Jun 02 20:33:15 +0000 2015,"{'first_person_level': 0, 'alcohol': 1, 'first...",0.523513,Beer Olympics with my #1 fan and babe😍❤️ #Team...,"{'friends_count': 356, 'followers_count': 563,...",Tue Dec 20 02:46:19 +0000 2011,11662,563,356,13940,False,3509904359376660926
556e1660d6dfbb462881361f,Tue Jun 02 20:48:04 +0000 2015,"{'first_person_level': 0, 'alcohol': 1, 'first...",0.536733,Drinking a Soft Parade by @ShortsBrewing @ Lak...,"{'friends_count': 626, 'followers_count': 543,...",Sun Mar 22 18:14:34 +0000 2009,13256,543,626,20051,False,5508552107751333732
556e16c2d6dfbb46288139a7,Tue Jun 02 20:49:44 +0000 2015,"{'first_person_level': 2, 'alcohol': 1, 'first...",0.761665,Its only Tuesday and I need a drink or four.,"{'friends_count': 516, 'followers_count': 211,...",Sun Jan 24 00:57:08 +0000 2010,1522,211,516,5300,False,-6198757192334168986
556e173dd6dfbb4628813e3b,Tue Jun 02 20:51:47 +0000 2015,"{'first_person_level': 2, 'alcohol': 1, 'first...",0.622639,jess told me no puking this friday.. so that m...,"{'friends_count': 408, 'followers_count': 4573...",Fri Dec 06 05:58:03 +0000 2013,6814,4573,408,7859,False,-3614958430482511601
556e1e58d6dfbb50e34a3eb7,Tue Jun 02 21:22:06 +0000 2015,"{'first_person_level': 1, 'alcohol': 1, 'first...",0.572562,"""I need a drink because I'm upset about some p...","{'friends_count': 624, 'followers_count': 727,...",Tue Jul 29 13:51:57 +0000 2008,2489,727,624,12889,False,-1823265372766523932


In [15]:
batch = X_fpl.sort_values(by='random_number').ix[:1000]

What was previously labeled as FPA

In [23]:
batch["text"] = batch.text.str.encode("utf-8")

In [25]:
date = str(datetime.date.today())
key = batch.ix[0].random_number
batch[["text", "random_number"]].to_csv("amt/c_amt {} {}.csv".format(date, key))

In [21]:
pickle.dump(batch, open('pickles/amt_test_batch.p', 'wb'))

In [212]:
datetime.datetime.now().time().minute

56

In [77]:
import os
def make_batch(pickled_df, size, output_dir):
    """
    pickled_df: unlabeled data with 'random_number' column
    size: # of items batch to be created
    output_dir: where the outputs are saved
    """
    # Load data
    X = pickle.load(open(pickled_df, 'rb'))
    
    # select size subset
    batch = X.sort_values(by='random_number').head(size)
    
    # process for turk
    batch["text"] = batch.text.str.encode("utf-8")
    date = str(datetime.date.today())
    time = datetime.datetime.now().time()
    key = batch.ix[0].random_number
    
    # persist
    description = "/c_amt {} {}h{}m {}".format(date, time.hour, time.minute, key)
    if not os.path.exists(output_dir + description):
        os.makedirs(output_dir + description)
    batch[["text", "random_number"]].to_csv(output_dir + description +  description + '.csv')
    pickle.dump(batch, open(output_dir + description + description +".p", 'wb'))
    X = X.drop(batch.index)
    pickle.dump(X, open(pickled_df, 'wb'))

In [266]:
make_batch('amt/unlabeled.p', 1000, 'amt')

In [259]:
X = DataAccess.get_as_dataframe()

In [260]:
X['random_number'] = X.text.apply(hash)

In [264]:
pickle.dump(X, open('amt/unlabeled.p', 'wb'))

In [204]:
def remove_dup(big_df, remove_these):
    return big_df.drop(remove_these.index)
    

In [268]:
test = pd.read_csv('C:/Users/Tom Work/PycharmProjects/nyu-twipsy/amt/c_amt 2016-06-02 16h30m -9217494627877178173/c_amt 2016-06-02 16h30m -9217494627877178173.csv')

In [274]:
batch = X.ix[test._id]

# Labeling the rest of the data

In [96]:
X = DataAccess.get_as_dataframe()
L = LabelGetter(X)
X_fpl, y = L.get_first_person_label()

In [65]:
old_results = pd.DataFrame.from_csv("C:/Users/Tom Work/Downloads/Batch_2417713_batch_results.csv")

In [72]:
old_results.index = old_results._id

In [99]:
%%time
X_fpl['random_number'] = X_fpl.text.apply(hash)

Wall time: 8 ms


In [98]:
X_fpl = X_fpl.ix[X_fpl.index.difference(old_results.index)]

In [101]:
X_fpl.shape

(5357, 12)

In [102]:
pickle.dump(X_fpl, open('amt/first_person_unlabeled.p', 'wb'))

In [104]:
make_batch('amt/first_person_unlabeled.p', 3000, 'amt')

In [105]:
batch1 = pd.DataFrame.from_csv("amt/c_amt 2016-06-03 11h16m -9223012266841220737/c_amt 2016-06-03 11h16m -9223012266841220737.csv")

In [106]:
batch1.shape

(3000, 2)

In [107]:
batch1.head()

,text,random_number
_id,,
558d9f98d6dfbb6950895bba,b'I will pay someone like $30 and a case of be...,-9223012266841220737
558f421bd6dfbb51bbe200d4,"b""@aileene_chew @dfts_90 @humankittehkat @va_k...",-9217032698021262422
55877c60d6dfbb344f78696f,b'Drinking a Stone Enjoy By 07.04.15 IPA by @S...,-9208811120167384204
5584d5b7d6dfbb6b2489b047,"b""Can't get enough of this cty. #chicago #rive...",-9207935440759435436
55789bb1d6dfbb052633defe,"b""If everyone would go out and have a drink to...",-9206623459859422642


In [108]:
batch2 = pd.DataFrame.from_csv("amt/c_amt 2016-06-03 11h16m 1125936180122922568/c_amt 2016-06-03 11h16m 1125936180122922568.csv")

In [109]:
batch2.shape

(2357, 2)

In [110]:
batch2.head()

,text,random_number
_id,,
5573b47ad6dfbb512f142635,"b""I'm in LA, at a gallery, watching a performa...",1125936180122922568
5587c02bd6dfbb46acebb825,b'Getting Drunk &amp; Fried With My Brother \x...,1130045892011414351
557639d8d6dfbb240cd14995,"b""I'm not a fan of Vodka""",1133567731290356678
55750755d6dfbb6738810e57,b'3 bottles of Jack. 1 ounce of weed. 1/2 song...,1134120824051744000
55721540d6dfbb33bf905f7d,b'Drinking a Marauder by @greatsouthbay at @ha...,1137236950417390560
